In [1]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


In [5]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)

# each review is encoded as a sequence of word indexes
# indexed by overall frequency in the dataset
# output is 0 (negative) or 1 (positive) 
imdb = tf.keras.datasets.imdb.load_data(num_words=max_features)
(raw_input_train, y_train), (raw_input_test, y_test) = imdb

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

input_train = tf.keras.preprocessing.sequence.pad_sequences(raw_input_train, maxlen=maxlen)
input_test = tf.keras.preprocessing.sequence.pad_sequences(raw_input_test, maxlen=maxlen)

In [7]:
input_train.shape, input_test.shape, y_train.shape, y_test.shape

((25000, 500), (25000, 500), (25000,), (25000,))

## LSTMs

In [8]:
embedding_dim = 32
dropout = 0.15
recurrent_dropout = 0.2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(name='embedding', input_dim=max_features, output_dim=embedding_dim, input_length=maxlen))

model.add(tf.keras.layers.LSTM(name='lstm', units=32))

# binary classifier
model.add(tf.keras.layers.Dense(name='fc', units=32, activation='relu'))
model.add(tf.keras.layers.Dense(name='classifier', units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
fc (Dense)                   (None, 32)                1056      
_________________________________________________________________
classifier (Dense)           (None, 1)                 33        
Total params: 329,409
Trainable params: 329,409
Non-trainable params: 0
_________________________________________________________________


In [9]:
batch_size = 1000

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 21s 1ms/step - loss: 0.6932 - acc: 0.5095 - val_loss: 0.6916 - val_acc: 0.5262
Epoch 2/10
20000/20000 [==============================] - 19s 934us/step - loss: 0.6876 - acc: 0.5590 - val_loss: 0.6855 - val_acc: 0.5564
Epoch 3/10
20000/20000 [==============================] - 19s 937us/step - loss: 0.6687 - acc: 0.6049 - val_loss: 0.6847 - val_acc: 0.5778
Epoch 4/10
20000/20000 [==============================] - 19s 933us/step - loss: 0.6375 - acc: 0.6357 - val_loss: 0.6212 - val_acc: 0.6614
Epoch 5/10
20000/20000 [==============================] - 16s 824us/step - loss: 0.5654 - acc: 0.7131 - val_loss: 0.5523 - val_acc: 0.7250
Epoch 6/10
20000/20000 [==============================] - 16s 807us/step - loss: 0.4822 - acc: 0.7748 - val_loss: 0.4782 - val_acc: 0.7816
Epoch 7/10
20000/20000 [==============================] - 16s 821us/step - loss: 0.4467 - acc: 0.7975 - val_loss: 0.519

In [10]:
train_loss, train_accuracy = model.evaluate(input_train, y_train, batch_size=batch_size)
train_accuracy

25000/25000 [==============================] - 5s 213us/step


0.8622000026702881

In [11]:
test_loss, test_accuracy = model.evaluate(input_test, y_test, batch_size=batch_size)
test_accuracy

25000/25000 [==============================] - 5s 214us/step


0.8226799988746643

In [12]:
# precition
model.predict(input_test[0:5])

array([[0.14266543],
       [0.96316284],
       [0.75627655],
       [0.84291   ],
       [0.9710915 ]], dtype=float32)

In [13]:
# ground truth
y_test[0:5]

array([0, 1, 1, 0, 1], dtype=int64)